# Langchain into

<img src="images/langchain_overview.png" width=800px />

## Компоненты LangChain

- **langchain-core**  
  Базовые абстракции для ключевых компонентов (модели, хранилища векторов, инструменты) и их компоновки. Легковесный, без сторонних интеграций.

- **langchain**  
  Общие цепочки и стратегии извлечения данных для построения когнитивной архитектуры приложений. Не привязан к сторонним системам.

- **Integration Packages**  
  Отдельные пакеты для популярных интеграций (например, `langchain-openai`, `langchain-anthropic`), чтобы обеспечить гибкость версий и минимальные зависимости.

- **langchain-community**  
  Сторонние интеграции, поддерживаемые сообществом. Включает интеграции для моделей, хранилищ и инструментов. Зависимости по умолчанию отключены.

- **langgraph**  
  Расширение для создания устойчивых, многоактных приложений с использованием графовой модели шагов.

- **langserve**  
  Инструмент для развертывания цепочек LangChain в виде REST API, подходящий для простых сценариев и известных примитивов.

- **LangSmith**  
  Платформа для разработчиков для отладки, тестирования, оценки и мониторинга LLM-приложений.

## 1. Langchain models


In [ ]:
# pip install langchain
# pip install langchain-openai

In [19]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv(override=True)

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o", verbose=True)

# Invoke the model with a message
result = model.invoke("Кто тебя создал?")
print("Full result:")
print(result)
print("Content only:")
print(result.content)

Full result:
content='Меня создала компания OpenAI.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'stop', 'logprobs': None} id='run-a04e30d2-eae3-438b-8188-48fec5479512-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Content only:
Меня создала компания OpenAI.


In [ ]:
from langchain.globals import set_verbose
from langchain.globals import set_debug
set_verbose(False)
set_debug(False)

messages = [
    SystemMessage(content="Ты - ИИ помощник, ты помогаешь людям."),
    HumanMessage(content="Давай познакомимся. Меня зовут Евгений, а тебя?"),
]


# Invoke the model with messages
ai_answer = model.invoke(messages)
print(type(ai_answer))
print(f"Answer from AI: {ai_answer.content}")


# AIMessage:
#   Message from an AI.
messages.append(ai_answer)
messages.append(HumanMessage(content="Напомни мне, как меня зовут?"))

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")


<class 'langchain_core.messages.ai.AIMessage'>
Answer from AI: Приятно познакомиться, Евгений! Я — твой виртуальный помощник, и, к сожалению, у меня нет имени. Как я могу помочь тебе сегодня?
Answer from AI: Конечно, ты сказал, что тебя зовут Евгений. Как я могу помочь тебе дальше?


In [ ]:
# pip install grpcio
# pip install yandexcloud
# pip install langchain-community

In [ ]:
from langchain_together import Together
from langchain_community.llms import YandexGPT
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


model = Together(model='Qwen/Qwen2.5-Coder-32B-Instruct', max_tokens=1024)
model = YandexGPT()

messages = [
    SystemMessage(content="Ты - ИИ помощник, ты помогаешь людям."),
    HumanMessage(content="Давай познакомимся. Меня зовут Евгений, а тебя?"),
]

# Invoke the model with messages
ai_answer = model.invoke(messages)
print(f"Answer from AI: {ai_answer}")


# AIMessage:
#   Message from an AI.
messages.append(AIMessage(content=ai_answer))
messages.append(HumanMessage(content="Напомни мне, как меня зовут?"))

# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result}")

Answer from AI: Меня зовут Алиса. Я могу ответить на ваши вопросы и помочь в решении различных задач. Чем я могу быть полезна?
Answer from AI: Вас зовут Евгений. Если у вас есть другие вопросы, я готова помочь.


## 2. Prompts templates

In [ ]:
from langchain.prompts import ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# # PART 1: Create a ChatPromptTemplate using a template string
template = "Придумай короткую шутку про {topic}."
prompt_template = ChatPromptTemplate.from_template(template)

print("-----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "крокодил"})
print(prompt)

# # PART 2: Prompt with Multiple Placeholders
template_multiple = """Ты - ИИ-помощник, ты помогаешь людям.
Human: Расскажи мне короткую {adjective} историю о {subject}."""
prompt_multiple = ChatPromptTemplate.from_template(template_multiple)
prompt = prompt_multiple.invoke({"adjective": "веселую", "subject": "крокодиле"})
print("\n----- Prompt with Multiple Placeholders -----\n")
print(prompt)


# PART 3: Prompt with System and Human Messages (Using Tuples)
messages = [
    ("system", "Ты выступаешь на стендапе и рассказываешь шутки на тему {topic}."),
    ("human", "Расскажи мне {joke_count} шуток."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "ИТ", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)

# # Extra Informoation about Part 4.
# # This does work:
# messages = [
#     ("system", "You are a comedian who tells jokes about {topic}."),
#     HumanMessage(content="Tell me 3 jokes."),
# ]
# prompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = prompt_template.invoke({"topic": "lawyers"})
# print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
# print(prompt)

## PART 4: Prompt with System and Human Messages templates
messages = [
SystemMessagePromptTemplate.from_template('Ты выступаешь на стендапе и рассказываешь шутки на тему {topic}.'),
HumanMessagePromptTemplate.from_template('Расскажи мне {joke_count} шуток.')
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "ИТ", "joke_count": 3})
print("\n----- Prompt with System and Human Messages templates -----\n")
print(prompt)


-----Prompt from Template-----
messages=[HumanMessage(content='Придумай короткую шутку про крокодил.', additional_kwargs={}, response_metadata={})]

----- Prompt with Multiple Placeholders -----

messages=[HumanMessage(content='Ты - ИИ-помощник, ты помогаешь людям.\nHuman: Расскажи мне короткую веселую историю о крокодиле.', additional_kwargs={}, response_metadata={})]

----- Prompt with System and Human Messages (Tuple) -----

messages=[SystemMessage(content='Ты выступаешь на стендапе и рассказываешь шутки на тему ИТ.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне 3 шуток.', additional_kwargs={}, response_metadata={})]

----- Prompt with System and Human Messages templates -----

messages=[SystemMessage(content='Ты выступаешь на стендапе и рассказываешь шутки на тему ИТ.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне 3 шуток.', additional_kwargs={}, response_metadata={})]


## 3. LCEL - LangChainExpression Language



In [43]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = YandexGPT()

# Define prompt templates (no need for separate Runnable chains)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты придумываешь шутки на тему {topic}."),
        ("human", "Напиши {joke_count} коротких смешных шутки."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()
# chain = prompt_template | model

# Run the chain
result = chain.invoke({"topic": "ИТ-разпработчики", "joke_count": 3})

# Output
print(result)

1. Встречаются два программиста на конференции. Один говорит другому: «Знаешь, как я называю код, который не могу понять? Это как иностранный язык, только вместо слов — цифры и символы».

2. Программист приходит на собеседование. Его спрашивают: «Какой самый популярный язык программирования вы знаете?» Он отвечает: «Английский».

3. Программисту говорят, что его код похож на спагетти. Он отвечает: «Ну, по крайней мере, он не сломается, как Java без точки с запятой».


In [45]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4")
model = YandexGPT()

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты придумываешь шутки на тему {topic}."),
        ("human", "Напиши {joke_count} коротких смешных шутки."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"topic": "сисадмины", "joke_count": 3})

# Output
print(response)

1. Сисадмин говорит коллеге: «У нас на сервере мышь завелась, всё время файлы грызёт!»

2. Встречаются два сисадмина, один у другого спрашивает: «Слушай, как ты так быстро всё в сети настраиваешь? Я вот как ни пытаюсь, ничего не получается...»
— А ты просто не забывай, что ты сисадмин, а не волшебник.

3. — Что такое сисадмин с похмелья?
— Это когда ты пытаешься зайти в свой кабинет, но тебя не пускает пароль «FAILED».


In [46]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")

# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Define additional processing steps using RunnableLambda
uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser() | uppercase_output | count_words

# Run the chain
result = chain.invoke({"topic": "lawyers", "joke_count": 3})

# Output
print(result)

Word count: 47
SURE, HERE ARE THREE LAWYER JOKES FOR YOU:

1. WHY DON'T SHARKS ATTACK LAWYERS?  
   PROFESSIONAL COURTESY.

2. WHAT’S THE DIFFERENCE BETWEEN A LAWYER AND A HERD OF BUFFALO?  
   THE LAWYER CHARGES MORE.

3. HOW MANY LAWYER JOKES ARE THERE, REALLY?  
   ONLY THREE. THE REST ARE TRUE STORIES!


In [58]:
# parallel chains

from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")
model = YandexGPT()

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert an cats and dogs. Answer in Russian."),
        ("human", "List the main features of the animal type {product_name}."),
    ]
)


# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert an cats and dogs"),
            (
                "human",
                "Given these features: {features}, list the 3 pros of these animal.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert an cats and dogs"),
            (
                "human",
                "Given these features: {features}, list the 3 cons of these animal.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

# Run the chain
result = chain.invoke({"product_name": "Кошка породы Корниш рекс"})

# Output
print(result)

Pros:
**Плюсы породы кошек корниш-рекс:**

1. **Оригинальный внешний вид.** Кошки корниш-рекс отличаются своей необычной кудрявой шерстью, которая придаёт им неповторимый вид. Это может быть привлекательным для тех, кто ищет что-то особенное среди пород кошек.

2. **Игривый и общительный характер.** Корниш-рексы очень активные и любопытные кошки, которые любят внимание и общение с людьми. Они могут стать отличными компаньонами для семей с детьми или для людей, которые ищут активного питомца.

3. **Отсутствие подшёрстка.** Из-за отсутствия подшёрстка, шерсть корниш-рексов не требует сложного ухода. Её достаточно регулярно расчёсывать мягкой щёткой или специальной перчаткой, чтобы удалить отмершие волоски. Это может облегчить уход за шерстью по сравнению с некоторыми другими породами кошек.

Однако стоит отметить, что корниш-рексы могут быть склонны к некоторым заболеваниям, поэтому важно следить за их здоровьем и регулярно посещать ветеринара. Также необходимо учитывать, что эти кошки м

In [57]:
# branches chains

from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")

model = YandexGPT()

# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer always in Russian."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | model | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | model | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

# Run the chain with an example review
# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is supergood"
result = chain.invoke({"feedback": review})

# Output the result
print(result)

Уважаемый [имя автора сообщения или название организации],

Я хотел(а) бы выразить свою благодарность за вашу помощь и внимание к моему предыдущему обращению. Я получил(а) положительный результат и остался(ась) доволен(льна) качеством предоставленной информации и уровнем обслуживания.

Однако у меня есть небольшой вопрос, который я хотел(а) уточнить. [Описание вопроса или проблемы]. Буду благодарен(льна), если вы сможете мне помочь.

Если это не входит в ваши обязанности, пожалуйста, сообщите мне, к кому я могу обратиться за дополнительной информацией или помощью.

Благодарю вас за ваше терпение и понимание.

С уважением,
[Ваше имя]


# 3. LangSmith

# 4. LangGraph